In [ ]:
!pip install transformers

In [ ]:
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder

import tensorflow as tf
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, Flatten, SimpleRNN
import torch
from transformers import BertForSequenceClassification, BertTokenizer, AdamW, get_linear_schedule_with_warmup

from sklearn.preprocessing import MinMaxScaler

from sklearn.metrics import roc_curve, auc
from transformers import BertTokenizer, TFBertForSequenceClassification

from google.colab import drive
drive.mount('/content/drive')

ValueError: mount failed

---
## Linux_Disk Neural Networks

In [ ]:
  class Model:
      def __init__(self):
          self.fcn = None
          self.rf = None

      def filter_scanning(self, df):
          #change K and M
          def convert_percentage_to_float(value):
            if value == '-':
                value = 0
            if isinstance(value, (float, int)):
                return float(value)
            if value.endswith('%'):
                value = float(value[:-1]) / 100
            return value
          def convert_to_float(value):
            if value == '-':
                value = 0
            if isinstance(value, (float, int)):
                return float(value)
            if value.endswith('K'):
                value = float(value[:-1]) * 1000
            elif value.endswith('M'):
                value = float(value[:-1]) * 1000000
            else:
                value = float(value)
            return value

          df['RDDSK'] = df['RDDSK'].apply(convert_to_float)
          df['WRDSK'] = df['WRDSK'].apply(convert_to_float)
          df['WCANCL'] = df['WCANCL'].apply(convert_to_float)

          df['DSK'] = df['DSK'].apply(convert_percentage_to_float)

          le = LabelEncoder()
          df['CMD'] = le.fit_transform(df['CMD'])

          scanning_count = df[df['type'] == 'scanning'].shape[0]

          # 2. Delete all rows that don't have 'scanning' or 'normal' in the 'type' column
          df = df[df['type'].isin(['scanning', 'normal'])]

          # Filter rows that have 'normal' in the 'type' column
          normal_rows = df[df['type'] == 'normal']

          # Calculate the number of rows to drop
          type_counts = df['type'].value_counts()
          rows_to_drop = type_counts['normal'] - scanning_count

          # Randomly select 'rows_to_drop' row indices to drop
          drop_indices = np.random.choice(normal_rows.index, rows_to_drop, replace=False)

          # Drop the selected rows from the DataFrame
          df = df.drop(drop_indices)

          df.drop(columns=['type', 'WCANCL'], inplace=True)

          return df

      def load_data(self, path):
          # Load the dataset
          df = pd.read_csv(path)
          df = self.filter_scanning(df)

          # Impute missing values with the mean value
          imputer = SimpleImputer(strategy='mean')
          df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

          # Split the dataset into training, validation, and testing sets
          X_trainval, X_test, y_trainval, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1],
                                                                    test_size=0.2, stratify=df.iloc[:, -1],
                                                                    random_state=42)
          X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval,
                                                            test_size=0.25, stratify=y_trainval,
                                                            random_state=42)

          # Normalize the numerical features
          scaler = StandardScaler()
          X_train = scaler.fit_transform(X_train)
          X_val = scaler.transform(X_val)
          X_test = scaler.transform(X_test)

          self.X_train, self.X_val, self.X_test = X_train, X_val, X_test
          self.y_train, self.y_val, self.y_test = y_train, y_val, y_test

      def train_cnn(self, epochs=10, batch_size=32):
          # Reshape the input for the CNN
          X_train_cnn = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
          X_val_cnn = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

          # Define the CNN architecture
          self.cnn = tf.keras.models.Sequential([
              Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
              MaxPooling1D(pool_size=2),
              Flatten(),
              Dense(1, activation='sigmoid')
          ])

          # Compile the CNN
          self.cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the CNN
          self.cnn.fit(X_train_cnn, self.y_train, validation_data=(X_val_cnn, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_cnn(self):
          # Reshape the input for the CNN
          X_test_cnn = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

          # Evaluate the CNN
          y_prob = self.cnn.predict(X_test_cnn)
          y_pred = np.round(y_prob).astype(int)
          print('CNN accuracy:', accuracy_score(self.y_test, y_pred))
          print('CNN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('CNN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

                  # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('CNN TPR:', tpr)
          print('CNN FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)
          np.save('CNN_fpr.npy', fpr)
          np.save('CNN_tpr.npy', tpr)

          roc_auc = auc(fpr, tpr)
          print('CNN ROC AUC:', roc_auc)

          idx = np.abs(fpr - 0.01).argmin()

          # Get the indices of fpr elements that are less than or equal to 0.01
          indices = np.where(fpr <= 0.01)[0]

          # Get the maximum tpr corresponding to fpr <= 0.01
          max_tpr = tpr[indices].max()

          # Get the index of the maximum tpr
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)
          print("Corresponding FPR: ", fpr[idx_max_tpr])

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 0.5])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

      def train_lstm(self, epochs=10, batch_size=32):
          # Reshape the input for the LSTM
          X_train_lstm = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
          X_val_lstm = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

          # Define the LSTM architecture
          self.lstm = tf.keras.models.Sequential([
              tf.keras.layers.LSTM(32, activation='relu', input_shape=(X_train_lstm.shape[1], 1)),
              tf.keras.layers.Dense(1, activation='sigmoid')
          ])

          # Compile the LSTM
          self.lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the LSTM
          self.lstm.fit(X_train_lstm, self.y_train, validation_data=(X_val_lstm, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_lstm(self):
          # Reshape the input for the LSTM
          X_test_lstm = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

          # Evaluate the LSTM
          y_prob = self.lstm.predict(X_test_lstm)
          y_pred = np.round(y_prob).astype(int)
          print('LSTM accuracy:', accuracy_score(self.y_test, y_pred))
          print('LSTM confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('LSTM classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('LSTM TPR:', tpr)
          print('LSTM FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)
          np.save('LSTM_fpr.npy', fpr)
          np.save('LSTM_tpr.npy', tpr)
          roc_auc = auc(fpr, tpr)
          print('LSTM ROC AUC:', roc_auc)

          idx = np.abs(fpr - 0.01).argmin()

          # Get the indices of fpr elements that are less than or equal to 0.01
          indices = np.where(fpr <= 0.01)[0]

          # Get the maximum tpr corresponding to fpr <= 0.01
          max_tpr = tpr[indices].max()

          # Get the index of the maximum tpr
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)
          print("Corresponding FPR: ", fpr[idx_max_tpr])

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 0.5])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

      def train_fcn(self, epochs=10, batch_size=32):
          # Define the FCN architecture
          self.fcn = tf.keras.models.Sequential([
              tf.keras.layers.Dense(32, activation='relu', input_shape=(self.X_train.shape[1],)),
              tf.keras.layers.Dense(1, activation='sigmoid')
          ])

          # Compile the FCN
          self.fcn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the FCN
          self.fcn.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_fcn(self):
          # Evaluate the FCN
          y_prob = self.fcn.predict(self.X_test)
          y_pred = np.round(y_prob).astype(int)
          print('FCN accuracy:', accuracy_score(self.y_test, y_pred))
          print('FCN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('FCN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('FCN TPR:', tpr)
          print('FCN FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)

          np.save('FCN_fpr.npy', fpr)
          np.save('FCN_tpr.npy', tpr)

          roc_auc = auc(fpr, tpr)
          print('FCN ROC AUC:', roc_auc)
          idx = np.abs(fpr - 0.01).argmin()
          indices = np.where(fpr <= 0.01)[0]
          max_tpr = tpr[indices].max()
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)
          print("Corresponding FPR: ", fpr[idx_max_tpr])

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 1.0])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

      def train_cnn_lstm(self, epochs=10, batch_size=32):
          # Reshape the input for the CNN-LSTM
          X_train_cnn_lstm = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
          X_val_cnn_lstm = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

          # Define the CNN-LSTM architecture
          self.cnn_lstm = tf.keras.models.Sequential([
              Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn_lstm.shape[1], 1)),
              MaxPooling1D(pool_size=2),
              LSTM(64),
              Dense(1, activation='sigmoid')
          ])

          # Compile the CNN-LSTM
          self.cnn_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the CNN-LSTM
          self.cnn_lstm.fit(X_train_cnn_lstm, self.y_train, validation_data=(X_val_cnn_lstm, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_cnn_lstm(self):
          # Reshape the input for the CNN-LSTM
          X_test_cnn_lstm = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

          # Evaluate the CNN-LSTM
          y_prob = self.cnn_lstm.predict(X_test_cnn_lstm)
          y_pred = np.round(y_prob).astype(int)
          print('CNN-LSTM accuracy:', accuracy_score(self.y_test, y_pred))
          print('CNN-LSTM confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('CNN-LSTM classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('CNN-LSTM TPR:', tpr)
          print('CNN-LSTM FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)

          np.save('CNN_LSTM_fpr.npy', fpr)
          np.save('CNN_LSTM_tpr.npy', tpr)

          roc_auc = auc(fpr, tpr)
          print('FCN ROC AUC:', roc_auc)
          idx = np.abs(fpr - 0.01).argmin()
          indices = np.where(fpr <= 0.01)[0]
          max_tpr = tpr[indices].max()
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)
          print("Corresponding FPR: ", fpr[idx_max_tpr])

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 1.0])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

      def train_rf(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1):
          # Define the Random Forest classifier
          self.rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                            min_samples_split=min_samples_split,
                                            min_samples_leaf=min_samples_leaf,
                                            random_state=42)

          # Train the Random Forest classifier
          self.rf.fit(self.X_train, self.y_train)

      def evaluate_rf(self):
          # Evaluate the Random Forest classifier
          y_pred = self.rf.predict(self.X_test)
          print('RF accuracy:', accuracy_score(self.y_test, y_pred))
          print('RF confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('RF classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('RF TPR:', tpr)
          print('RF FPR:', fpr)

      def train_rnn(self, epochs=10, batch_size=32):
          # Reshape the input for the RNN
          X_train_rnn = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
          X_val_rnn = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

          # Define the RNN architecture
          self.rnn = tf.keras.models.Sequential([
              SimpleRNN(32, activation='relu', input_shape=(X_train_rnn.shape[1], 1)),
              Dense(1, activation='sigmoid')
          ])

          # Compile the RNN
          self.rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the RNN
          self.rnn.fit(X_train_rnn, self.y_train, validation_data=(X_val_rnn, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_rnn(self):
          # Reshape the input for the RNN
          X_test_rnn = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

          # Evaluate the RNN
          y_prob = self.rnn.predict(X_test_rnn)
          y_pred = np.round(y_prob).astype(int)
          print('RNN accuracy:', accuracy_score(self.y_test, y_pred))
          print('RNN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('RNN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('RNN TPR:', tpr)
          print('RNN FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)
          np.save('RNN_fpr.npy', fpr)
          np.save('RNN_tpr.npy', tpr)

          roc_auc = auc(fpr, tpr)
          print('FCN ROC AUC:', roc_auc)

          idx = np.abs(fpr - 0.01).argmin()

          # Get the indices of fpr elements that are less than or equal to 0.05
          indices = np.where(fpr <= 0.01)[0]

          # Get the maximum tpr corresponding to fpr <= 0.01
          max_tpr = tpr[indices].max()

          # Get the index of the maximum tpr
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 1.0])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

      def train_ann(self, epochs=10, batch_size=32):
          # Define the ANN architecture
          self.ann = tf.keras.models.Sequential([
              tf.keras.layers.Dense(32, activation='relu', input_shape=(self.X_train.shape[1],)),
              tf.keras.layers.Dense(64, activation='relu'),
              tf.keras.layers.Dense(1, activation='sigmoid')
          ])

          # Compile the ANN
          self.ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

          # Train the ANN
          self.ann.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs=epochs, batch_size=batch_size)

      def evaluate_ann(self):
          # Evaluate the ANN
          y_prob = self.ann.predict(self.X_test)
          y_pred = np.round(y_prob).astype(int)
          print('ANN accuracy:', accuracy_score(self.y_test, y_pred))
          print('ANN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
          print('ANN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

          # Calculate TPR and FPR
          cm = confusion_matrix(self.y_test, y_pred)
          tn, fp, fn, tp = cm.ravel()
          tpr = tp / (tp + fn)
          fpr = fp / (fp + tn)
          print('ANN TPR:', tpr)
          print('ANN FPR:', fpr)

          # Calculate ROC AUC
          fpr, tpr, _ = roc_curve(self.y_test, y_prob)
          np.save('ANN_fpr.npy', fpr)
          np.save('ANN_tpr.npy', tpr)
          roc_auc = auc(fpr, tpr)
          print('FCN ROC AUC:', roc_auc)
          idx = np.abs(fpr - 0.01).argmin()
          indices = np.where(fpr <= 0.01)[0]
          max_tpr = tpr[indices].max()
          idx_max_tpr = np.where(tpr == max_tpr)[0][0]

          print("Maximum TPR with FPR <= 0.01: ", max_tpr)
          print("Corresponding FPR: ", fpr[idx_max_tpr])

          # Plot ROC curve
          plt.figure()
          plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
          plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
          plt.xlim([0.0, 0.5])
          plt.ylim([0.0, 1.05])
          plt.xlabel('False Positive Rate')
          plt.ylabel('True Positive Rate')
          plt.title('Receiver Operating Characteristic')
          plt.legend(loc="lower right")
          plt.show()

In [ ]:
clf=Model()

In [ ]:
clf.load_data('drive/MyDrive/ToN_IoT/Train_Test_datasets/Train_Test_Linux_dataset/linux_disk.csv')

<ipython-input-19-37c9fa9d8f6e>:62: DtypeWarning: Columns (2,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [ ]:
clf.train_cnn()
clf.evaluate_cnn()

#TPR after 0.01FPR: 0.20515099223468508

In [ ]:
clf.train_lstm()
clf.evaluate_lstm()

#TPR: 0.979025435
#TPR after 0.01 FPR rule: 0.48090046278139464

In [ ]:
clf.train_fcn()
clf.evaluate_fcn()

#FCN TPR: 0.979370931053416
#Maximum TPR with FPR <= 0.01:  0.26194132873166525

In [ ]:
clf.train_cnn_lstm()
clf.evaluate_cnn_lstm()

#CNN-LSTM TPR: 0.9710565534551729 (8.2%)
#Maximum TPR with FPR <= 0.01:  0.5407082908463409

In [ ]:
clf.train_rnn()
clf.evaluate_rnn()

#RNN TPR: 0.9675268648521452 (6% FPR)
#Maximum TPR with FPR <= 0.01:  0.6173817554317985

In [ ]:
clf.train_ann()
clf.evaluate_ann()

#ANN TPR: 0.9871362459800769
#Maximum TPR with FPR <= 0.01:  0.4516401286375402

In [ ]:
clf.train_rf()
clf.evaluate_rf()

#RF TPR: 0.9967840614950192

---
## Linux_Process Neural Networks

In [ ]:
class Model:
    def __init__(self):
        self.fcn = None
        self.rf = None

    def preprocess_linux_process(self, df):
        le = LabelEncoder()
        df['POLI'] = le.fit_transform(df['POLI'])
        df['Status'] = le.fit_transform(df['Status'])
        df['State'] = le.fit_transform(df['State'])
        df['CMD'] = le.fit_transform(df['CMD'])

        scanning_count = df[df['type'] == 'scanning'].shape[0]

        # 2. Delete all rows that don't have 'scanning' or 'normal' in the 'type' column
        df = df[df['type'].isin(['scanning', 'normal'])]

        # Filter rows that have 'normal' in the 'type' column
        normal_rows = df[df['type'] == 'normal']

        # Calculate the number of rows to drop
        type_counts = df['type'].value_counts()
        rows_to_drop = type_counts['normal'] - scanning_count

        # Randomly select 'rows_to_drop' row indices to drop
        drop_indices = np.random.choice(normal_rows.index, rows_to_drop, replace=False)

        # Drop the selected rows from the DataFrame
        df = df.drop(drop_indices)

        df.drop(columns=['type'], inplace=True)

        return df

    def load_data(self, path):
        # Load the dataset
        df = pd.read_csv(path)
        df = self.preprocess_linux_process(df)

        # Impute missing values with the mean value
        imputer = SimpleImputer(strategy='mean')
        df = pd.DataFrame(imputer.fit_transform(df), columns=df.columns)

        # Split the dataset into training, validation, and testing sets
        X_trainval, X_test, y_trainval, y_test = train_test_split(df.iloc[:, :-1], df.iloc[:, -1],
                                                                  test_size=0.2, stratify=df.iloc[:, -1],
                                                                  random_state=42)
        X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval,
                                                          test_size=0.25, stratify=y_trainval,
                                                          random_state=42)

        # Normalize the numerical features
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_val = scaler.transform(X_val)
        X_test = scaler.transform(X_test)

        self.X_train, self.X_val, self.X_test = X_train, X_val, X_test
        self.y_train, self.y_val, self.y_test = y_train, y_val, y_test
    def train_cnn(self, epochs=10, batch_size=32):
        # Reshape the input for the CNN
        X_train_cnn = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
        X_val_cnn = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

        # Define the CNN architecture
        self.cnn = tf.keras.models.Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn.shape[1], 1)),
            MaxPooling1D(pool_size=2),
            Flatten(),
            Dense(1, activation='sigmoid')
        ])

        # Compile the CNN
        self.cnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the CNN
        self.cnn.fit(X_train_cnn, self.y_train, validation_data=(X_val_cnn, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_cnn(self):
        # Reshape the input for the CNN
        X_test_cnn = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

        # Evaluate the CNN
        y_prob = self.cnn.predict(X_test_cnn)
        y_pred = np.round(y_prob).astype(int)
        print('CNN accuracy:', accuracy_score(self.y_test, y_pred))
        print('CNN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('CNN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

                # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('CNN TPR:', tpr)
        print('CNN FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('CNN_fpr.npy', fpr)
        np.save('CNN_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('CNN ROC AUC:', roc_auc)

        idx = np.abs(fpr - 0.01).argmin()

        # Get the indices of fpr elements that are less than or equal to 0.01
        indices = np.where(fpr <= 0.01)[0]

        # Get the maximum tpr corresponding to fpr <= 0.01
        max_tpr = tpr[indices].max()

        # Get the index of the maximum tpr
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    def train_lstm(self, epochs=10, batch_size=32):
        # Reshape the input for the LSTM
        X_train_lstm = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
        X_val_lstm = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

        # Define the LSTM architecture
        self.lstm = tf.keras.models.Sequential([
            tf.keras.layers.LSTM(32, activation='relu', input_shape=(X_train_lstm.shape[1], 1)),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        # Compile the LSTM
        self.lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the LSTM
        self.lstm.fit(X_train_lstm, self.y_train, validation_data=(X_val_lstm, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_lstm(self):
        # Reshape the input for the LSTM
        X_test_lstm = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

        # Evaluate the LSTM
        y_prob = self.lstm.predict(X_test_lstm)
        y_pred = np.round(y_prob).astype(int)
        print('LSTM accuracy:', accuracy_score(self.y_test, y_pred))
        print('LSTM confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('LSTM classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('LSTM TPR:', tpr)
        print('LSTM FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('LSTM_fpr.npy', fpr)
        np.save('LSTM_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('LSTM ROC AUC:', roc_auc)

        idx = np.abs(fpr - 0.01).argmin()

        # Get the indices of fpr elements that are less than or equal to 0.01
        indices = np.where(fpr <= 0.01)[0]

        # Get the maximum tpr corresponding to fpr <= 0.01
        max_tpr = tpr[indices].max()

        # Get the index of the maximum tpr
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    def train_fcn(self, epochs=10, batch_size=32):
        # Define the FCN architecture
        self.fcn = tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(self.X_train.shape[1],)),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        # Compile the FCN
        self.fcn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the FCN
        self.fcn.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_fcn(self):
        # Evaluate the FCN
        y_prob = self.fcn.predict(self.X_test)
        y_pred = np.round(y_prob).astype(int)
        print('FCN accuracy:', accuracy_score(self.y_test, y_pred))
        print('FCN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('FCN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('FCN TPR:', tpr)
        print('FCN FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('FCN_fpr.npy', fpr)
        np.save('FCN_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('FCN ROC AUC:', roc_auc)
        idx = np.abs(fpr - 0.01).argmin()
        indices = np.where(fpr <= 0.01)[0]
        max_tpr = tpr[indices].max()
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    def train_cnn_lstm(self, epochs=10, batch_size=32):
        # Reshape the input for the CNN-LSTM
        X_train_cnn_lstm = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
        X_val_cnn_lstm = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

        # Define the CNN-LSTM architecture
        self.cnn_lstm = tf.keras.models.Sequential([
            Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(X_train_cnn_lstm.shape[1], 1)),
            MaxPooling1D(pool_size=2),
            LSTM(64),
            Dense(1, activation='sigmoid')
        ])

        # Compile the CNN-LSTM
        self.cnn_lstm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the CNN-LSTM
        self.cnn_lstm.fit(X_train_cnn_lstm, self.y_train, validation_data=(X_val_cnn_lstm, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_cnn_lstm(self):
        # Reshape the input for the CNN-LSTM
        X_test_cnn_lstm = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

        # Evaluate the CNN-LSTM
        y_prob = self.cnn_lstm.predict(X_test_cnn_lstm)
        y_pred = np.round(y_prob).astype(int)
        print('CNN-LSTM accuracy:', accuracy_score(self.y_test, y_pred))
        print('CNN-LSTM confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('CNN-LSTM classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('CNN-LSTM TPR:', tpr)
        print('CNN-LSTM FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('CNN_LSTM_fpr.npy', fpr)
        np.save('CNN_LSTM_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('FCN ROC AUC:', roc_auc)
        idx = np.abs(fpr - 0.01).argmin()
        indices = np.where(fpr <= 0.01)[0]
        max_tpr = tpr[indices].max()
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    def train_rf(self, n_estimators=100, max_depth=None, min_samples_split=2, min_samples_leaf=1):
        # Define the Random Forest classifier
        self.rf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth,
                                          min_samples_split=min_samples_split,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=42)

        # Train the Random Forest classifier
        self.rf.fit(self.X_train, self.y_train)

    def evaluate_rf(self):
        # Evaluate the Random Forest classifier
        y_pred = self.rf.predict(self.X_test)
        print('RF accuracy:', accuracy_score(self.y_test, y_pred))
        print('RF confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('RF classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('RF TPR:', tpr)
        print('RF FPR:', fpr)

    def train_rnn(self, epochs=10, batch_size=32):
        # Reshape the input for the RNN
        X_train_rnn = self.X_train.reshape(self.X_train.shape[0], self.X_train.shape[1], 1)
        X_val_rnn = self.X_val.reshape(self.X_val.shape[0], self.X_val.shape[1], 1)

        # Define the RNN architecture
        self.rnn = tf.keras.models.Sequential([
            SimpleRNN(32, activation='relu', input_shape=(X_train_rnn.shape[1], 1)),
            Dense(1, activation='sigmoid')
        ])

        # Compile the RNN
        self.rnn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the RNN
        self.rnn.fit(X_train_rnn, self.y_train, validation_data=(X_val_rnn, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_rnn(self):
        # Reshape the input for the RNN
        X_test_rnn = self.X_test.reshape(self.X_test.shape[0], self.X_test.shape[1], 1)

        # Evaluate the RNN
        y_prob = self.rnn.predict(X_test_rnn)
        y_pred = np.round(y_prob).astype(int)
        print('RNN accuracy:', accuracy_score(self.y_test, y_pred))
        print('RNN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('RNN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('RNN TPR:', tpr)
        print('RNN FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('RNN_fpr.npy', fpr)
        np.save('RNN_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('FCN ROC AUC:', roc_auc)
        idx = np.abs(fpr - 0.011).argmin()
        indices = np.where(fpr <= 0.011)[0]
        max_tpr = tpr[indices].max()
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()

    def train_ann(self, epochs=10, batch_size=32):
        # Define the ANN architecture
        self.ann = tf.keras.models.Sequential([
            tf.keras.layers.Dense(32, activation='relu', input_shape=(self.X_train.shape[1],)),
            tf.keras.layers.Dense(64, activation='relu'),
            tf.keras.layers.Dense(1, activation='sigmoid')
        ])

        # Compile the ANN
        self.ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

        # Train the ANN
        self.ann.fit(self.X_train, self.y_train, validation_data=(self.X_val, self.y_val), epochs=epochs, batch_size=batch_size)

    def evaluate_ann(self):
        # Evaluate the ANN
        y_prob = self.ann.predict(self.X_test)
        y_pred = np.round(y_prob).astype(int)
        print('ANN accuracy:', accuracy_score(self.y_test, y_pred))
        print('ANN confusion matrix:\n', confusion_matrix(self.y_test, y_pred))
        print('ANN classification report:\n', classification_report(self.y_test, y_pred, digits=4))

        # Calculate TPR and FPR
        cm = confusion_matrix(self.y_test, y_pred)
        tn, fp, fn, tp = cm.ravel()
        tpr = tp / (tp + fn)
        fpr = fp / (fp + tn)
        print('ANN TPR:', tpr)
        print('ANN FPR:', fpr)

        # Calculate ROC AUC
        fpr, tpr, _ = roc_curve(self.y_test, y_prob)
        np.save('ANN_fpr.npy', fpr)
        np.save('ANN_tpr.npy', tpr)
        roc_auc = auc(fpr, tpr)
        print('FCN ROC AUC:', roc_auc)
        idx = np.abs(fpr - 0.01).argmin()
        indices = np.where(fpr <= 0.01)[0]
        max_tpr = tpr[indices].max()
        idx_max_tpr = np.where(tpr == max_tpr)[0][0]

        print("Maximum TPR with FPR <= 0.01: ", max_tpr)
        print("Corresponding FPR: ", fpr[idx_max_tpr])

        # Plot ROC curve
        plt.figure()
        plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
        plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        plt.xlim([0.0, 1.0])
        plt.ylim([0.0, 1.05])
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title('Receiver Operating Characteristic')
        plt.legend(loc="lower right")
        plt.show()


In [ ]:
clf = Model()
clf.load_data('drive/MyDrive/ToN_IoT/Train_Test_datasets/Train_Test_Linux_dataset/linux_process.csv')

In [ ]:
clf.train_cnn()
clf.evaluate_cnn()

In [ ]:
clf.train_lstm()
clf.evaluate_lstm()

In [ ]:
clf.train_fcn()
clf.evaluate_fcn()

#FCN TPR: 0.9899869960988297 (0.09% FPR)
#Maximum TPR with FPR <= 0.01:  0.9260078023407022

In [ ]:
clf.train_cnn_lstm()
clf.evaluate_cnn_lstm()

#CNN-LSTM TPR: 0.9947984395318595 (10 Percent FPR)
#Maximum TPR with FPR <= 0.01:  0.9433029908972692

In [ ]:
clf.train_rnn()
clf.evaluate_rnn()

#RNN TPR: 0.9849154746423927 (0.015%)
#Maximum TPR with FPR <= 0.01:  0.9837451235370611

In [ ]:
clf.train_ann()
clf.evaluate_ann()

#ANN TPR: 0.9876462938881665 (8.4%)
#Maximum TPR with FPR <= 0.01:  0.9677503250975292

In [ ]:
clf.train_rf()
clf.evaluate_rf()

#RF TPR: 0.9942782834850455